In [43]:
import matplotlib.pyplot as plt
import pickle as pkl
import numpy as np
import scipy.signal as ss
import torch
import os
import warnings
warnings.filterwarnings("ignore")

from weight import array_weight_vector
from utils import *
from doa import music
from models import Model1
sim_tm = Time()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

repo_dir = os.getcwd()
tm = Time()

In [44]:
device

device(type='cuda')

## Sample Location Generation For Testing

In [45]:
%matplotlib notebook
np.random.seed(6)

P,Q = 8,8 # rows and columns of antenna array
lamda = 1 # wavelength
ue = 1  # number of user equipments
ris = 1  # number of RIS
n_source = ue+ris
n_antenna = P*Q  # number of antenna elements 
snr = 10

print('\nsimulation sample data\n----------------------')
array = np.linspace(0,(n_antenna-1)/2,n_antenna)

# angle for RIS would be added
p_thetas = (np.pi/3)*(np.random.rand(n_source))   # random source polar directions
az_thetas = (2*np.pi)*(np.random.rand(n_source))   # random source azimuthal directions

# power from RIS would be added
ue_alphas = np.sqrt(1/2)*(np.random.randn(ue) + np.random.randn(ue)*1j) # random UE powers
ris_alphas = np.sqrt(1/2)*(np.random.randn(ris) + np.random.randn(ris)*1j) # random RIS powers

print('random signal direction #polar:',p_thetas*180/np.pi)
print('random signal direction #azimuthal:',az_thetas*180/np.pi)

ue_vectors, ris_vectors = [],[]
ue_angles, ris_angles = [], []
for polar,az in zip(p_thetas[:ue],az_thetas[:ue]):
    ue_vectors.append(find_position_vector(polar,az))
    ue_angles.append([polar*180/np.pi,az*180/np.pi])

for polar,az in zip(p_thetas[ue:],az_thetas[ue:]):
    ris_vectors.append(find_position_vector(polar,az))
    ris_angles.append([polar*180/np.pi,az*180/np.pi])

bs_vector = [0,0,0]
print('\nbase station locations:',bs_vector)
print(f'\nRIS \ndirection vectors: {ris_vectors}\ndirection angles: {ris_angles}\npower: {ris_alphas}')
print(f'\nuser equipment \ndirection vectors: {ue_vectors}\ndirection angles: {ue_angles}\npower: {ue_alphas}')


simulation sample data
----------------------
random signal direction #polar: [53.57160909 19.91878832]
random signal direction #azimuthal: [295.6424843   15.01078526]

base station locations: [0, 0, 0]

RIS 
direction vectors: [[0.006067509241204117, 2.7744162567926823e-05, 0.9999815921115095]]
direction angles: [[19.918788318070632, 15.010785261089966]]
power: [-1.75841946+0.64576634j]

user equipment 
direction vectors: [[0.016251992379168842, 0.0014675892375824373, 0.999866850598387]]
direction angles: [[53.571609086160095, 295.64248430081943]]
power: [0.15402256-0.63575391j]


## Sample Data Generation

In [46]:
numAngles = 360
p_angles = np.linspace(0,np.pi/3,numAngles)*180/np.pi
az_angles = np.linspace(0,2*np.pi,numAngles)*180/np.pi
general_angles = np.array(np.meshgrid(p_angles,az_angles)).T.reshape(-1, 2).tolist()
print('Total sample location collected:', len(general_angles))

Total sample location collected: 129600


# Deep Learning

In [47]:
sim_tm.start(message="processing simulation #deep-learning")

In [48]:
numrealization = 100 # number of time samples collected at antenna array
tm.start(message='preparing covariance matrix by DL')
H = np.zeros((n_antenna,numrealization)) + 1j*np.zeros((n_antenna,numrealization))
print('H matrix shape(n_antenna,time_instance):',H.shape)
antenna_weight = array_weight_vector(ris_vectors,
                                     ue_vectors,
                                     ris_angles,
                                     ue_angles,
                                     vector=True,
                                     angle=True,)

for iter in range(numrealization):
    # random distortions due to propagation medium
    #ris
    distortion_ris = np.exp(1j*2*np.pi*np.random.rand(1)) 
    recieved_power_ris = distortion_ris*ris_alphas*antenna_weight
    #ue
    distortion_ue = np.exp(1j*2*np.pi*np.random.rand(1))
    recieved_power_ue = distortion_ue*ue_alphas*antenna_weight
    
    net_recieved_power = recieved_power_ris+recieved_power_ue
    noise = np.sqrt(0.5/snr)*(np.random.randn(n_antenna)+np.random.randn(n_antenna)*1j)
    H[:,iter] = net_recieved_power+noise
CovMat = H@H.conj().transpose()
print('covariance matrix shape:',CovMat.shape)
tm.end()

H matrix shape(n_antenna,time_instance): (64, 100)
covariance matrix shape: (64, 64)

>> preparing covariance matrix by DL: Done!! Time taken: 0.0138 sec


In [49]:
%%time

# MUSIC algorithm

# DoAsMUSIC, psindB = music(CovMat,
#                           L = n_source,
#                           N = n_antenna,
#                           angles = general_angles,
#                           ris_data = ris_angles+ris_vectors,
#                           height=None,
#                           method='dl')

CPU times: total: 0 ns
Wall time: 0 ns


In [50]:
file_path = os.path.join(repo_dir,'Data','psindB.pkl')
# with open(file_path, 'wb') as file:
#     pkl.dump(psindB, file)
    
with open(file_path, 'rb') as file:
    psindB = pkl.load(file)

In [51]:
%%time
signal_samples = 10
DoAsMUSIC,_= ss.find_peaks(psindB)
indies = np.argsort(psindB[DoAsMUSIC])[-1*signal_samples:]
samples = np.array(general_angles)[DoAsMUSIC][indies]

sinr = SINR(ris_power = ris_alphas,
            ue_power = ue_alphas,
            ue_angles = samples,
            ris_data = ris_angles+ris_vectors,
            snr = 10)
detected_ue_angles = samples[np.argmax(sinr)].tolist()
print('detected angles for user equipment:',detected_ue_angles)
print('actual UE direction:',ue_angles)
print('with maximum SINR:',np.max(sinr))
print()

detected angles for user equipment: [50.64066852367687, 284.7910863509749]
actual UE direction: [[53.571609086160095, 295.64248430081943]]
with maximum SINR: 19.86365324043524

CPU times: total: 1.86 s
Wall time: 155 ms


In [52]:
%matplotlib inline

frequency = 3e8
duration = 5
sampling_rate=50 
speed= frequency*lamda
ue_angle = samples[np.argmax(sinr)].tolist()
specified_vector = find_position_vector(ue_angle[0],ue_angle[1])
ris_data=ris_angles+ris_vectors
weights = array_weight_vector(ris_vectors=[ris_data[1]],
                              ue_vectors=[specified_vector],
                              ris_angles=[ris_data[0]],
                              ue_angles=[ue_angle],
                              vector=True,
                              angle=True
                             )

#ris
distortion_ris = np.exp(1j*2*np.pi*np.random.default_rng(seed=42).random(1)) 
recieved_power_ris = np.sum(distortion_ris*ris_alphas*weights)/n_antenna
ris_time, ris_sinusoidal_wave = complex_to_sinusoidal(recieved_power_ris, 
                                                      frequency, 
                                                      duration,
                                                      sampling_rate=sampling_rate)

#ue
distortion_ue = np.exp(1j*2*np.pi*np.random.default_rng(seed=21+np.argmax(sinr)).random(1))
recieved_power_ue = np.sum(distortion_ue*ue_alphas*weights)/n_antenna
ue_time, ue_sinusoidal_wave = complex_to_sinusoidal(recieved_power_ue, 
                                                    frequency, 
                                                    duration,
                                                    sampling_rate=sampling_rate)

print('RIS\n---')
print('amplitude:',np.abs(recieved_power_ris))
phase_ris = np.angle(recieved_power_ris)*180/np.pi
print('phase:',phase_ris)

print('\nUser Equipement\n---------------')
print('amplitude:',np.abs(recieved_power_ue))
phase_ue = np.angle(recieved_power_ue)*180/np.pi
print('phase:',phase_ue)

phase_diff = abs(phase_ris-phase_ue)
omega = 360*frequency
del_t = phase_diff/omega
del_d = speed*del_t

print(f'\ntime delay: {abs(del_t)} seconds')
print(f'∇d: {del_d} meters')

RIS
---
amplitude: 1.3278521844202702
phase: 121.81296752118382

User Equipement
---------------
amplitude: 0.4636912856726687
phase: 143.94829709885073

time delay: 2.0495675534876768e-10 seconds
∇d: 0.061487026604630304 meters


In [53]:
sim_tm.end()


>> processing simulation #deep-learning: Done!! Time taken: 0.2501 sec


# Neural Network (Model 1)

In [54]:
model = Model1(input_size=4).to(device=device)

input_ = 'azimuthal'
model_name = 'model1'
weight_file = os.listdir(os.path.join(repo_dir,'model_states',input_, model_name))[0]
state = torch.load(os.path.join(repo_dir,'model_states',input_,model_name,weight_file), 
                   map_location=torch.device(device))

model.load_state_dict(state)
print(model.eval())

loc = 70
cols = [0,1,5,6] 
# position: [2,3,4,7,8,9] or angle: [0,1,5,6] or position+angle: range(10)

vector_check = False
angle_check = True
neural_network=True # make it false when using LSTM or related models

Model1(
  (linear1): Linear(in_features=4, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=256, bias=True)
  (linear4): Linear(in_features=256, out_features=512, bias=True)
  (linear5): Linear(in_features=512, out_features=256, bias=True)
  (linear6): Linear(in_features=256, out_features=128, bias=True)
  (linear7): Linear(in_features=128, out_features=128, bias=True)
)


In [55]:
sim_tm.start(message="processing simulation #neural-network")

In [56]:
numrealization = 100 # number of time samples collected at antenna array
H = np.zeros((n_antenna,numrealization)) + 1j*np.zeros((n_antenna,numrealization))
print('H matrix shape(n_antenna,time_instance):',H.shape)
antenna_weight = array_weight_vector(ris_vectors,
                                     ue_vectors,
                                     ris_angles,
                                     ue_angles,
                                     vector=vector_check,
                                     angle=angle_check,
                                     model=model,
                                     neural_network=neural_network) 

for iter in range(numrealization):
    # random distortions due to propagation medium
    #ris
    distortion_ris = np.exp(1j*2*np.pi*np.random.rand(1)) 
    recieved_power_ris = distortion_ris*ris_alphas*antenna_weight
    #ue
    distortion_ue = np.exp(1j*2*np.pi*np.random.rand(1))
    recieved_power_ue = distortion_ue*ue_alphas*antenna_weight
    
    net_recieved_power = recieved_power_ris+recieved_power_ue
    noise = np.sqrt(0.5/snr)*(np.random.randn(n_antenna)+np.random.randn(n_antenna)*1j)
    H[:,iter] = net_recieved_power+noise
CovMat = H@H.conj().transpose()
print('covariance matrix shape:',CovMat.shape)

H matrix shape(n_antenna,time_instance): (64, 100)
covariance matrix shape: (64, 64)


In [57]:
%%time

# MUSIC algorithm
# DoAsMUSIC, psindB = music(CovMat,
#                           L = n_source,
#                           N = n_antenna,
#                           angles = general_angles,
#                           ris_data = ris_angles+ris_vectors,
#                           model = model,
#                           vector_ = vector_check,
#                           angle_ = angle_check,
#                           height=[7, 9],
#                           method='dl',
#                           P=P,
#                           Q=Q,
#                           lamda=lamda,
#                           neural_network=neural_network
#                          )

CPU times: total: 0 ns
Wall time: 0 ns


In [58]:
file_path = os.path.join(repo_dir,'Data','psindB-NN.pkl')
# with open(file_path, 'wb') as file:
#     pkl.dump(psindB, file)
    
with open(file_path, 'rb') as file:
    psindB = pkl.load(file)

In [59]:
%%time
signal_samples = 10
DoAsMUSIC,_= ss.find_peaks(psindB)
indies = np.argsort(psindB[DoAsMUSIC])[-1*signal_samples:]
samples = np.array(general_angles)[DoAsMUSIC][indies]

sinr = SINR(ris_power = ris_alphas,
            ue_power = ue_alphas,
            ue_angles = samples,
            ris_data = ris_angles+ris_vectors,
            model = model,
            vector = vector_check,
            angle = angle_check,
            snr = 10,
            neural_network=neural_network)

detected_ue_angles = samples[np.argmax(sinr)].tolist()
print('detected angles for user equipment:',detected_ue_angles)
print('actual UE direction:',ue_angles)
print('with maximum SINR:',np.max(sinr))
print()

detected angles for user equipment: [54.3175487465181, 299.8328690807799]
actual UE direction: [[53.571609086160095, 295.64248430081943]]
with maximum SINR: 42.8588202394683

CPU times: total: 750 ms
Wall time: 66.3 ms


In [60]:
%matplotlib inline

frequency = 3e8
duration = 5
sampling_rate=50 
speed= frequency*lamda
ue_angle = samples[np.argmax(sinr)].tolist()
specified_vector = find_position_vector(ue_angle[0],ue_angle[1])
ris_data=ris_angles+ris_vectors
weights = array_weight_vector(ris_vectors=[ris_data[1]],
                              ue_vectors=[specified_vector],
                              ris_angles=[ris_data[0]],
                              ue_angles=[ue_angle],
                              vector=vector_check,
                              angle=angle_check,
                              model=model,
                              neural_network=neural_network)     

#ris
distortion_ris = np.exp(1j*2*np.pi*np.random.default_rng(seed=42).random(1)) 
recieved_power_ris = np.sum(distortion_ris*ris_alphas*weights)/n_antenna
ris_time, ris_sinusoidal_wave = complex_to_sinusoidal(recieved_power_ris, 
                                                      frequency, 
                                                      duration,
                                                      sampling_rate=sampling_rate)

#ue
distortion_ue = np.exp(1j*2*np.pi*np.random.default_rng(seed=21+np.argmax(sinr)).random(1))
recieved_power_ue = np.sum(distortion_ue*ue_alphas*weights)/n_antenna
ue_time, ue_sinusoidal_wave = complex_to_sinusoidal(recieved_power_ue, 
                                                    frequency, 
                                                    duration,
                                                    sampling_rate=sampling_rate)

print('RIS\n---')
print('amplitude:',np.abs(recieved_power_ris))
phase_ris = np.angle(recieved_power_ris)*180/np.pi
print('phase:',phase_ris)

print('\nUser Equipement\n---------------')
print('amplitude:',np.abs(recieved_power_ue))
phase_ue = np.angle(recieved_power_ue)*180/np.pi
print('phase:',phase_ue)

phase_diff = abs(phase_ris-phase_ue)
omega = 360*frequency
del_t = phase_diff/omega
del_d = speed*del_t

print(f'\ntime delay: {abs(del_t)} seconds')
print(f'∇d: {del_d} meters')

RIS
---
amplitude: 264.9338420960134
phase: 133.22591882577447

User Equipement
---------------
amplitude: 92.51595569226293
phase: 110.27052762122315

time delay: 2.1254991856066035e-10 seconds
∇d: 0.0637649755681981 meters


In [61]:
sim_tm.end()


>> processing simulation #neural-network: Done!! Time taken: 0.1324 sec


# NSB Algorithm

In [62]:
sim_tm.start(message="processing simulation #nsb")

In [63]:
numrealization = 100 # number of time samples collected at antenna array
tm.start(message='preparing covariance matrix by NSB')
H = np.zeros((n_antenna,numrealization)) + 1j*np.zeros((n_antenna,numrealization))
print('H matrix shape(n_antenna,time_instance):',H.shape)
antenna_weight = nsb(P,Q, wavelength=lamda, pos_angles=ris_angles+ue_angles).reshape(-1,)

for iter in range(numrealization):
    # random distortions due to propagation medium
    #ris
    distortion_ris = np.exp(1j*2*np.pi*np.random.rand(1)) 
    recieved_power_ris = distortion_ris*ris_alphas*antenna_weight
    #ue
    distortion_ue = np.exp(1j*2*np.pi*np.random.rand(1))
    recieved_power_ue = distortion_ue*ue_alphas*antenna_weight
    
    net_recieved_power = recieved_power_ris+recieved_power_ue
    noise = np.sqrt(0.5/snr)*(np.random.randn(n_antenna)+np.random.randn(n_antenna)*1j)
    H[:,iter] = net_recieved_power+noise
CovMat = H@H.conj().transpose()
print('covariance matrix shape:',CovMat.shape)
tm.end()

H matrix shape(n_antenna,time_instance): (64, 100)
covariance matrix shape: (64, 64)

>> preparing covariance matrix by NSB: Done!! Time taken: 0.0087 sec


In [64]:
%%time

# MUSIC algorithm

# DoAsMUSIC, psindB = music(CovMat,
#                           L = n_source,
#                           N = n_antenna,
#                           angles = general_angles,
#                           ris_data = ris_angles+ris_vectors,
#                           height=None,
#                           method='nsb')

CPU times: total: 0 ns
Wall time: 0 ns


In [65]:
file_path = os.path.join(repo_dir,'Data','psindB-NSB.pkl')
# with open(file_path, 'wb') as file:
#     pkl.dump(psindB, file)
    
with open(file_path, 'rb') as file:
    psindB = pkl.load(file)

In [66]:
%%time
signal_samples = 10
DoAsMUSIC,_= ss.find_peaks(psindB)
indies = np.argsort(psindB[DoAsMUSIC])[-1*signal_samples:]
samples = np.array(general_angles)[DoAsMUSIC][indies]

sinr = SINR(ris_power = ris_alphas,
            ue_power = ue_alphas,
            ue_angles = samples,
            ris_data = ris_angles+ris_vectors,
            snr = 10)
detected_ue_angles = samples[np.argmax(sinr)].tolist()
print('detected angles for user equipment:',detected_ue_angles)
print('actual UE direction:',ue_angles)
print('with maximum SINR:',np.max(sinr))
print()

detected angles for user equipment: [50.47353760445682, 166.46239554317546]
actual UE direction: [[53.571609086160095, 295.64248430081943]]
with maximum SINR: 19.941919289584963

CPU times: total: 1.31 s
Wall time: 114 ms


In [67]:
%time

frequency = 3e8 #40e9  
speed= frequency*lamda
duration = 5
sampling_rate=50 # >2*frequency
ue_angle = samples[np.argmax(sinr)].tolist()
ris_data=ris_angles+ris_vectors
weights = nsb(P,Q, wavelength=lamda, pos_angles=[ris_data[0]]+[ue_angle]).reshape(-1,)

#ris
distortion_ris = np.exp(1j*2*np.pi*np.random.default_rng(seed=42).random(1)) 
recieved_power_ris = np.sum(distortion_ris*ris_alphas*weights)/n_antenna
ris_time, ris_sinusoidal_wave = complex_to_sinusoidal(recieved_power_ris, 
                                                      frequency, 
                                                      duration,
                                                      sampling_rate=sampling_rate)

#ue
distortion_ue = np.exp(1j*2*np.pi*np.random.default_rng(seed=21+np.argmax(sinr)).random(1))
recieved_power_ue = np.sum(distortion_ue*ue_alphas*weights)/n_antenna
ue_time, ue_sinusoidal_wave = complex_to_sinusoidal(recieved_power_ue, 
                                                    frequency, 
                                                    duration,
                                                    sampling_rate=sampling_rate)

print('RIS\n---')
print('amplitude:',np.abs(recieved_power_ris))
phase_ris = np.angle(recieved_power_ris)*180/np.pi
print('phase:',phase_ris)

print('\nUser Equipement\n---------------')
print('amplitude:',np.abs(recieved_power_ue))
phase_ue = np.angle(recieved_power_ue)*180/np.pi
print('phase:',phase_ue)

phase_diff = abs(phase_ris-phase_ue)
omega = 360*frequency
del_t = phase_diff/omega
del_d = speed*del_t

print(f'\ntime delay: {abs(del_t)} seconds')
print(f'∇d: {del_d} meters')

CPU times: total: 0 ns
Wall time: 0 ns
RIS
---
amplitude: 0.0050600973838557965
phase: 161.35846518356462

User Equipement
---------------
amplitude: 0.0017670062143049366
phase: 138.4030739790133

time delay: 2.1254991856066035e-10 seconds
∇d: 0.0637649755681981 meters


In [68]:
sim_tm.end()


>> processing simulation #nsb: Done!! Time taken: 0.1777 sec


# Capon Method

In [69]:
sim_tm.start(message="processing simulation #capon")

In [70]:
M = P*Q # number of antenna elements (sensors)
pos_angles = ris_angles+ue_angles
Rss = np.eye((2))# correlation matrix of the information symbols
Rnn = 0.1*np.eye((M))# correlation matrix of additive noise
A = find_steering_matrix(wavelength=lamda, pos_angles=pos_angles, planar_antenna_shape=(P,Q)) # steering vectors
R = A @ Rss @ np.conj(A).T + Rnn # correlation matrix
g = np.array([1,1]) # gate array: both DoAs are "switched on"

In [71]:
numrealization = 100 # number of time samples collected at antenna array
tm.start(message='preparing covariance matrix by Capon')
H = np.zeros((n_antenna,numrealization)) + 1j*np.zeros((n_antenna,numrealization))
print('H matrix shape(n_antenna,time_instance):',H.shape)
antenna_weight = capon(A,R,g)

for iter in range(numrealization):
    # random distortions due to propagation medium
    #ris
    distortion_ris = np.exp(1j*2*np.pi*np.random.rand(1)) 
    recieved_power_ris = distortion_ris*ris_alphas*antenna_weight
    #ue
    distortion_ue = np.exp(1j*2*np.pi*np.random.rand(1))
    recieved_power_ue = distortion_ue*ue_alphas*antenna_weight
    
    net_recieved_power = recieved_power_ris+recieved_power_ue
    noise = np.sqrt(0.5/snr)*(np.random.randn(n_antenna)+np.random.randn(n_antenna)*1j)
    H[:,iter] = net_recieved_power+noise
CovMat = H@H.conj().transpose()
print('covariance matrix shape:',CovMat.shape)
tm.end()

H matrix shape(n_antenna,time_instance): (64, 100)
covariance matrix shape: (64, 64)

>> preparing covariance matrix by Capon: Done!! Time taken: 0.0060 sec


In [72]:
%%time

# MUSIC algorithm

# DoAsMUSIC, psindB = music(CovMat,
#                           L = n_source,
#                           N = n_antenna,
#                           angles = general_angles,
#                           ris_data = ris_angles+ris_vectors,
#                           height=None,
#                           method='capon')

CPU times: total: 0 ns
Wall time: 0 ns


In [73]:
file_path = os.path.join(repo_dir,'Data','psindB-Capon.pkl')
# with open(file_path, 'wb') as file:
#     pkl.dump(psindB, file)
    
with open(file_path, 'rb') as file:
    psindB = pkl.load(file)

In [74]:
%%time
signal_samples = 10
DoAsMUSIC,_= ss.find_peaks(psindB)
indies = np.argsort(psindB[DoAsMUSIC])[-1*signal_samples:]
samples = np.array(general_angles)[DoAsMUSIC][indies]

sinr = SINR(ris_power = ris_alphas,
            ue_power = ue_alphas,
            ue_angles = samples,
            ris_data = ris_angles+ris_vectors,
            snr = 10)
detected_ue_angles = samples[np.argmax(sinr)].tolist()
print('detected angles for user equipment:',detected_ue_angles)
print('actual UE direction:',ue_angles)
print('with maximum SINR:',np.max(sinr))
print()

detected angles for user equipment: [19.38718662952646, 18.050139275766014]
actual UE direction: [[53.571609086160095, 295.64248430081943]]
with maximum SINR: 19.93052052968314

CPU times: total: 1.67 s
Wall time: 145 ms


In [75]:
%time

frequency = 3e8  
speed= frequency*lamda
duration = 5
sampling_rate=50 # >2*frequency
ue_angle = samples[np.argmax(sinr)].tolist()
ris_data=ris_angles+ris_vectors
pos_angles = [ris_data[0]]+[ue_angle]
A = find_steering_matrix(wavelength=lamda, pos_angles=pos_angles, planar_antenna_shape=(P,Q)) # steering vectors
R = A @ Rss @ np.conj(A).T + Rnn # correlation matrix
g = np.array([1,1]) # gate array: both DoAs are "switched on"
weights = capon(A,R,g)

#ris
distortion_ris = np.exp(1j*2*np.pi*np.random.default_rng(seed=42).random(1)) 
recieved_power_ris = np.sum(distortion_ris*ris_alphas*weights)/n_antenna
ris_time, ris_sinusoidal_wave = complex_to_sinusoidal(recieved_power_ris, 
                                                      frequency, 
                                                      duration,
                                                      sampling_rate=sampling_rate)

#ue
distortion_ue = np.exp(1j*2*np.pi*np.random.default_rng(seed=21+np.argmax(sinr)).random(1))
recieved_power_ue = np.sum(distortion_ue*ue_alphas*weights)/n_antenna
ue_time, ue_sinusoidal_wave = complex_to_sinusoidal(recieved_power_ue, 
                                                    frequency, 
                                                    duration,
                                                    sampling_rate=sampling_rate)

print('RIS\n---')
print('amplitude:',np.abs(recieved_power_ris))
phase_ris = np.angle(recieved_power_ris)*180/np.pi
print('phase:',phase_ris)

print('\nUser Equipement\n---------------')
print('amplitude:',np.abs(recieved_power_ue))
phase_ue = np.angle(recieved_power_ue)*180/np.pi
print('phase:',phase_ue)

phase_diff = abs(phase_ris-phase_ue)
omega = 360*frequency
del_t = phase_diff/omega
del_d = speed*del_t

print(f'\ntime delay: {abs(del_t)} seconds')
print(f'∇d: {del_d} meters')

CPU times: total: 0 ns
Wall time: 0 ns
RIS
---
amplitude: 0.004673095319066681
phase: 155.49114669609588

User Equipement
---------------
amplitude: 0.0016318635477600234
phase: 177.6264762737628

time delay: 2.0495675534876768e-10 seconds
∇d: 0.061487026604630304 meters


In [76]:
sim_tm.end()


>> processing simulation #capon: Done!! Time taken: 0.2152 sec


# Bartlett's Method

In [77]:
sim_tm.start(message="processing simulation #bartlett")

In [78]:
M = P*Q # number of antenna elements (sensors)
A = find_steering_matrix(wavelength=lamda, pos_angles=pos_angles, planar_antenna_shape=(P,Q)) # steering vectors

In [79]:
numrealization = 100 # number of time samples collected at antenna array
tm.start(message='preparing covariance matrix by Bartlett')
H = np.zeros((n_antenna,numrealization)) + 1j*np.zeros((n_antenna,numrealization))
print('H matrix shape(n_antenna,time_instance):',H.shape)
antenna_weight = np.average(A,axis=1)/M

for iter in range(numrealization):
    # random distortions due to propagation medium
    #ris
    distortion_ris = np.exp(1j*2*np.pi*np.random.rand(1)) 
    recieved_power_ris = distortion_ris*ris_alphas*antenna_weight
    #ue
    distortion_ue = np.exp(1j*2*np.pi*np.random.rand(1))
    recieved_power_ue = distortion_ue*ue_alphas*antenna_weight
    
    net_recieved_power = recieved_power_ris+recieved_power_ue
    noise = np.sqrt(0.5/snr)*(np.random.randn(n_antenna)+np.random.randn(n_antenna)*1j)
    H[:,iter] = net_recieved_power+noise
CovMat = H@H.conj().transpose()
print('covariance matrix shape:',CovMat.shape)
tm.end()

H matrix shape(n_antenna,time_instance): (64, 100)
covariance matrix shape: (64, 64)

>> preparing covariance matrix by Bartlett: Done!! Time taken: 0.0063 sec


In [80]:
%%time

# MUSIC algorithm

# DoAsMUSIC, psindB = music(CovMat,
#                           L = n_source,
#                           N = n_antenna,
#                           angles = general_angles,
#                           ris_data = ris_angles+ris_vectors,
#                           height=None,
#                           method='bartlett')

CPU times: total: 0 ns
Wall time: 0 ns


In [81]:
file_path = os.path.join(repo_dir,'Data','psindB-Bartlett.pkl')
# with open(file_path, 'wb') as file:
#     pkl.dump(psindB, file)
    
with open(file_path, 'rb') as file:
    psindB = pkl.load(file)

In [82]:
%%time
signal_samples = 10
DoAsMUSIC,_= ss.find_peaks(psindB)
indies = np.argsort(psindB[DoAsMUSIC])[-1*signal_samples:]
samples = np.array(general_angles)[DoAsMUSIC][indies]

sinr = SINR(ris_power = ris_alphas,
            ue_power = ue_alphas,
            ue_angles = samples,
            ris_data = ris_angles+ris_vectors,
            snr = 10)
detected_ue_angles = samples[np.argmax(sinr)].tolist()
print('detected angles for user equipment:',detected_ue_angles)
print('actual UE direction:',ue_angles)
print('with maximum SINR:',np.max(sinr))
print()

detected angles for user equipment: [13.370473537604456, 351.9777158774373]
actual UE direction: [[53.571609086160095, 295.64248430081943]]
with maximum SINR: 19.86133481190123

CPU times: total: 1.12 s
Wall time: 98.1 ms


In [83]:
%time

frequency = 3e8 #40e9  
speed= frequency*lamda
duration = 5
sampling_rate=50 # >2*frequency
ue_angle = samples[np.argmax(sinr)].tolist()
ris_data=ris_angles+ris_vectors

pos_angles = [ris_data[0]]+[ue_angle]
A = find_steering_matrix(wavelength=lamda, pos_angles=pos_angles, planar_antenna_shape=(P,Q)) # steering vectors
weights = np.average(A,axis=1)/M

#ris
distortion_ris = np.exp(1j*2*np.pi*np.random.default_rng(seed=42).random(1)) 
recieved_power_ris = np.sum(distortion_ris*ris_alphas*weights)/n_antenna
ris_time, ris_sinusoidal_wave = complex_to_sinusoidal(recieved_power_ris, 
                                                      frequency, 
                                                      duration,
                                                      sampling_rate=sampling_rate)

#ue
distortion_ue = np.exp(1j*2*np.pi*np.random.default_rng(seed=21+np.argmax(sinr)).random(1))
recieved_power_ue = np.sum(distortion_ue*ue_alphas*weights)/n_antenna
ue_time, ue_sinusoidal_wave = complex_to_sinusoidal(recieved_power_ue, 
                                                    frequency, 
                                                    duration,
                                                    sampling_rate=sampling_rate)

print('RIS\n---')
print('amplitude:',np.abs(recieved_power_ris))
phase_ris = np.angle(recieved_power_ris)*180/np.pi
print('phase:',phase_ris)

print('\nUser Equipement\n---------------')
print('amplitude:',np.abs(recieved_power_ue))
phase_ue = np.angle(recieved_power_ue)*180/np.pi
print('phase:',phase_ue)

phase_diff = abs(phase_ris-phase_ue)
omega = 360*frequency
del_t = phase_diff/omega
del_d = speed*del_t

# time_delay = find_time_delay(ue_sinusoidal_wave, ris_sinusoidal_wave, sampling_rate)

print(f'\ntime delay: {abs(del_t)} seconds')
print(f'∇d: {del_d} meters')

CPU times: total: 0 ns
Wall time: 0 ns
RIS
---
amplitude: 0.003613831771880581
phase: 175.23559135332638

User Equipement
---------------
amplitude: 0.001261964486837549
phase: -162.62907906900674

time delay: 3.1283765779845657e-09 seconds
∇d: 0.9385129733953698 meters


In [84]:
sim_tm.end()


>> processing simulation #bartlett: Done!! Time taken: 0.1728 sec
